In [ ]:
# Imports

## For AOV

import torch
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F

# For Displaying and Saving Results

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(42) # the answer to the universe

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = {
    'training':datasets.MNIST('mnist-monkfromearth/dataset/training/', download=True, train=True, transform=transform),
    'testing':datasets.MNIST('mnist-monkfromearth/dataset/testing/', download=True, train=False, transform=transform)
}

dataloader = {
    'training':torch.utils.data.DataLoader(dataset['training'], batch_size=64, shuffle=True),
    'testing':torch.utils.data.DataLoader(dataset['testing'], batch_size=64, shuffle=True)
}

In [ ]:
# For showing images

def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

In [ ]:
# Getting a sample image

images, labels = next(iter(dataloader['training']))
imshow(images[0])

In [ ]:
# Creating a model, criterion

model = nn.Sequential(
    nn.Linear(784, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 10)
)

criterion = nn.CrossEntropyLoss()

In [ ]:
learning_rates = np.arange(0.001, 0.011, step=0.001, dtype=np.float64)
momentums = np.arange(0.003, 0.01, step=0.003, dtype=np.float64)

total = len(learning_rates) * len(momentums)

data = pd.DataFrame(columns=['Learning Rate', 'Momentum', 'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10','Accuracy'], index=range(1,total+1))

In [ ]:
from itertools import product

steps = 0

for rate, momentum in list(product(learning_rates, momentums)):
    optimizer = optim.SGD(model.parameters(), lr=rate, momentum=momentum)
    
    data.iloc[steps][0] = rate
    data.iloc[steps][1] = momentum
    
    print("Learning Rate: {}, Momentum: {}".format(rate, momentum))
    
    training_losses = []
    test_losses = []
    
    epochs = 10
    for e in range(epochs):
        training_loss = 0
        test_loss = 0
        accuracy = 0
        
        for images, labels in dataloader['training']:
            images = images.view(images.shape[0], -1)
            scores = model(images)
            optimizer.zero_grad()
            loss = criterion(scores, labels)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
        else:
            model.eval()
            with torch.no_grad():
                for images, labels in dataloader['testing']:
                    images = images.view(images.shape[0], -1)
                    scores = model(images)
                    loss = criterion(scores, labels)
                    test_loss += loss.item()
                    ps = torch.exp(scores)
                    top_p, top_class = ps.topk(1, dim=1)
                    equality = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equality.type(torch.FloatTensor))
            current_training_loss = training_loss / len(dataloader['training'])
            current_test_loss = test_loss / len(dataloader['testing'])
            current_accuracy = accuracy / len(dataloader['testing']) 
        model.train()
        training_losses.append(current_training_loss)
        test_losses.append(current_test_loss)
        data.iloc[steps][e + 2] = current_accuracy
        print("Epoch: {}, Training Loss: {}, Test Loss: {}, Accuracy: {}".format(e, current_training_loss, current_test_loss, current_accuracy))
    print()
    steps += 1
    fig = plt.figure(figsize=(11,6))
    plt.plot(train_losses, label='Training loss')
    plt.plot(test_losses, label='Validation loss')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.title("Rate:{}, Momentum:{}".format(rate, momentum))
    plt.legend()
    plt.show()
    fig.savefig("mnist-monkfromearth/results/LearningRate_{}_Momentum_{}.png".format(rate, momentum))

In [ ]:
data